In [136]:
import numpy as np
import string
import random 
import nltk
import numpy as num 
import tensorflow as tf
import keras
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import BayesianOptimization
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '10'
from bs4 import BeautifulSoup
import requests
import re
import translators as ts
import majka
import json
import pickle

In [137]:
seed = 1998
np.random.seed(seed)
tf.random.set_seed(seed)

In [138]:
def get_menza(url = "https://www.vse.cz/menza/stravovani-zizkov/"):
    
    table = BeautifulSoup(requests.get(url).content, "html.parser").find("table", class_ = "menza-table")

    foods = []

    item_foods = []

    for row in table.find_all('tr'):
        for item in row.find_all('td'):
            item_food =  str(item.find_all("div") ).replace('<div>','').replace('</div>','').replace('[', '').replace(']','')
            item_foods.append(item_food)

    for item in item_foods:
    # Split the string based on commas and numbers in parentheses
        split_items = re.split(r',\s*(?=\d+\,)|,\s*(?=\d+\))', item)
        for sub_item in split_items:
        # Remove any standalone numbers with commas
            cleaned_food_item = re.sub(r'^\d+(?=,?\s*)|\d+(?=,?\s*$)', '', sub_item)
            cleaned_food_item = re.sub(r'^[\s,]+', '', cleaned_food_item)
            foods.append(cleaned_food_item.strip())
    """
    for ind, i in enumerate(foods):
        try:
            if i[-1].isdigit():
                foods[ind] = i[:-1]
        except:
            continue
    """

    final_chalky = [i for i in foods if i != '' and len(i) > 1]

    return final_chalky

In [139]:
intents = {"intents": [
            
              {"tag": "bakalářský program",
               "patterns": ["Jaké tu jsou bakalářské programy?", "bakalářský program", "bakalářské programy",
                            "bakalářský studijní program", "bakalářské studijní programy", "bakalář", "Bc."
                            "Jaké tu jsou bakalářské studijní programy?",
                            "Jaké bakalářské programy tu mohu studovat?"],
               "responses": ["Na FIS VŠE jsou vyučovány tyto bakalářské programy: Aplikovaná informatika; Data Analytics; Informační média a služby; Matematické metody v ekonomii; Multimédia v ekonomické praxi"]
              },
               {"tag": "magisterský program",
               "patterns": ["Jaké tu jsou magisterské programy?", "magisterský program", "magisterské programy", "magisterský studijní program", "magisterské studijní programy", "magistr", "Ing.",
                            "Jaké tu jsou navazující magisterské programy?", "navazující magisterský program", "navazující magisterské programy", "navazující magisterský studijní program", "navazující magisterské studijní programy", "navazující magistr",],
               "responses": ["Na FIS VŠE jsou vyučovány tyto magisterské programy: Informační management; Business Intelligence (IST); Data a analytika pro business; Ekonometrie a operační výzkum; Ekonomická demografie; Statistika; Business analýza (IST); Kognitivní informatika; Řízení podnikové informatiky (IST); UX výzkum a design (IST); Vývoj informačních systémů (IST); Znalostní a webové technologie,; Information Systems Management; Economic Data Analysis"]
              },
              {"tag": "vedlejší specializace",
               "patterns": ["Jaké tu jsou vedlejší specializace?", "vedlejší specializace", "jaké jsou vedlejší specializace", "vedlejška", "vedlejšky"],
               "responses": ["Na FIS VŠE jsou vyučovány tyto vedlejší specializace: Analýzy sociálně ekonomických dat; Datové inženýrství; Demografie a sociální analýza; Ekonometrie; Informatika v podnikání; Inteligentní systémy; Komunikace, intepretace, multimédia; Kvantitativní analýza; Kvantitativní metody pro informatiky; Kvantitativní metody v managementu; Multimediální komunikace; Pojistné inženýrství; Prezentace a komunikace informací; Řízení podnikové výkonnosti; Security Management"]
                
              },
              {"tag": "doktorský program",
               "patterns": ["Jaké tu jsou doktorské programy?", "doktorský program", "doktorské programy", "doktorský studijní program", "doktorské studijní programy", "doktor", "doktorand", "doktorandský", "PhD", "Ph.D.",
                            "Jaké tu jsou doktorandské programy?", "doktorandský program", "doktorandské programy", "doktorandský studijní program", "doktorandské studijní programy",],
               "responses": ["Na FIS VŠE jsou vyučovány tyto doktorské programy: Aplikovaná informatika; Ekonometrie a operační výzkum; Statistika"]
                
              },

               {"tag": "bakalářský předmět Multimédia v ekonomické praxi",
               "patterns": ["bakalářský předmět Multimédia v ekonomické praxi", "bakalářské předměty Multimédia v ekonomické praxi",
                            "bakalářský kurz Multimédia v ekonomické praxi", "bakalářské kurzy Multimédia v ekonomické praxi", "Multimédia v ekonomické praxi",
                            "Jaké jsou povinné předměty na bakalářském programu Multimédia v ekonomické praxi?"],
               "responses": ["Na bakalářském programu Multimédia v ekonomické praxi jsou vyučovány tyto povinné předměty: 3D grafika; Design komunikací; Digitální fotografie; Grafika a animace; Multimediální projektový seminář; Multimediální technologie; Počítačová grafika; Praxe v multimédiích; Prezentační dovednosti; Profese v audiovizuální výrobě; Projekt audiovizuální komunikace; Reklamní agentura; Řízení multimediálních projektů (kreativita a komunikace); Sémiotika multimédií; Tvůrčí zpracování jazyka a textu; Úvod do multimédií; Web design a uživatelská rozhraní; Základy audiovizuální komunikace; Základy digitální fotografie; Základy počítačové grafiky; Zvuk a multimédia"]
              },

               {"tag": "bakalářský předmět Data Analytics",
               "patterns": ["bakalářský předmět Data Analytics", "bakalářské předměty Data Analytics", "bakalářský kurz Data Analytics",
                            "bakalářské kurzy Data Analytics", "Data Analytics",
                            "Jaké jsou povinné předměty na bakalářském programu Data Analytics?"
                            ],
               "responses": ["Na bakalářském programu Data Analytics jsou vyučovány tyto povinné předměty: Data mining v cloudu; Databáze; Datová kvalita; Datové minimum; Externí data a informační zdroje; Nástroje pro datovou analytiku; Neuronové sítě a deep learning; Principy digitální komunikace; Řízení datového projektu; Strojové učení 1; Strojové učení 2; Textová analytika 1; Textová analytika 2; Úvod do programování v jazyce Python; Úvod do programování v R; Základy Business Intelligence; Zpracování Big Data; Deskripce, vizualizace a komparace ekonomických dat; Ekonometrické modely; Exploratorní analýza dat; Matematická informatika; Matematická statistika; Matematika pro informatiky; Optimalizační modely; Pravděpodobnost a náhodné procesy; Průzkumy a dotazníková šetření; Regresní modely; Bankovnictví a finanční instituce; Ekonomie 1; Kritické a analytické myšlení; Prezentace a storytelling; Účetnictví I.; Základy informační etiky a práve; Základy marketingu pro informatiky a statistiky; Základy odborné práce"]
              },

               {"tag": "bakalářský předmět Aplikovaná informatika",
               "patterns": ["bakalářský předmět Aplikovaná informatika", "bakalářské předměty Multimédia v ekonomické praxi",
                            "bakalářský kurz Aplikovaná informatika", "bakalářské kurzy Aplikovaná informatika", "Aplikovaná informatika",
                            "Jaké jsou povinné předměty na bakalářském programu Aplikovaná informatika?"],
               "responses": ["Na bakalářském programu Aplikovaná informatika jsou vyučovány tyto povinné předměty: Analýza a návrh informačních systémů; Bakalářský seminář; Bezpečnost informačních systémů; Databáze; Datové minimum; Informační a komunikační technologie; IT Governance; Kvantitativní management; Podnikové informační systémy; Programování v Javě; Simulační modely pro informatiky; Softwarové inženýrství; UX design; Základy odborné práce; Zpracování informací a znalostí; Ekomomie 1; Finance podniku; Finanční teorie, politika a instituce; Management pro informatiky a statistiky; Matematika pro informatiky; Právo; Statistika pro informatiky; Strategická analýza pro informatiky a statistiky; Účetnictví I.; Základy marketingu pro informatiky a statistiky"]
              },

               {"tag": "bakalářský předmět Informační média a služby",
               "patterns": ["bakalářský předmět Informační média a služby", "bakalářské předměty Informační média a služby", "bakalářský kurz Informační média a služby",
                            "bakalářské kurzy Informační média a služby", "Informační média a služby",
                            "Jaké jsou povinné předměty na bakalářském programu Informační média a služby?"],
               "responses": ["Na bakalářském programu Informační média a služby jsou vyučovány tyto povinné předměty: Bakalářský seminář; Bezpečnost informačních systémů; Databáze; Datové minimum; Digitální knihovny a archivy; Informace a společnost; Informační architektura; Informační a komunikační technologie; Integrace informačních zdrojů; Matematické základy informatiky; Obchodní a firemní informace; Organizace znalostí; Programování v Javě; Psychologie kreativity; Rešeršní strategie a systémy; Sémiotika multimédií; Teorie a praxe týmové práce; Základy odborné práce; Zpracování informací a znalostí; Ekonomie 1; Finance podniku; Matematika pro informatiky; Právo; Statistika pro informatiky; Strategická analýza pro informatiky a statistiky; Účetnictví I.; Základy marketingu pro informatiky a statistiky"]
              },

               {"tag": "bakalářský předmět Matematické metody v ekonomii",
               "patterns": ["bakalářský předmět Matematické metody v ekonomii", "bakalářské předměty Matematické metody v ekonomii", "bakalářský kurz Matematické metody v ekonomii",
                            "bakalářské kurzy Matematické metody v ekonomii", "Matematické metody v ekonomii",
                            "Jaké jsou povinné předměty na bakalářském programu Matematické metody v ekonomii?"],
               "responses": ["Na bakalářském programu Matematické metody v ekonomii jsou vyučovány tyto povinné předměty: Demografické projekce; Matematika 2; Metody statistického srovnávání; Modely operačního výzkumu; Statistické metody 1; Stochastické modely; Úvod do programování v R; Úvod do teorie pravděpodobnosti a matematické statistiky; Základy datových analýz; Základy demografie; Ekonomie 1; Finanční teorie, politika a instituce; Informatika pro statistiky; Management pro informatiky a statistiky; Matematika pro ekonomy; Právo; Účetnictví I.; Základy marketingu pro informatiky a statistiky ... Další povinné předměty se pak liší dle vybrané specializace"]
              },

               {"tag": "bakalářský předmět Ekonomická demografie",
               "patterns": ["bakalářský předmět Ekonomická demografie", "bakalářské předměty Ekonomická demografie", "bakalářský kurz Ekonomická demografie", "bakalářské kurzy Ekonomická demografie", "Ekonomická demografie", "Matematické metody v ekonomii"
                            "specializace Ekonomická demografie (Matematické metody v ekonomii)",
                            "Jaké jsou povinné předměty na specializaci Ekonomická demografie (Matematické metody v ekonomii)?"],
               "responses": ["Na specializaci Ekonomická demografie (součást bakalářského programu Matematické metody v ekonomii) jsou vyučovány tyto povinné předměty: Bakalářský seminář; Demografická praktika pro statistiky; Ekonomická demografie I; Hospodářská a sociální statistika; Hospodářská politika pro statistiky; Regionální demografie; Statistické metody 2; Sociální politika; Úvod do sociálně ekonomické demografie; Zdroje a databáze demografických dat"]
              },

               {"tag": "bakalářský předmět Datové analýzy a modelování",
               "patterns": ["bakalářský předmět Datové analýzy a modelování", "bakalářské předměty Datové analýzy a modelování", "bakalářský kurz Datové analýzy a modelování", "bakalářské kurzy Datové analýzy a modelování", "Datové analýzy a modelování", "Matematické metody v ekonomii"
                            "specializace Datové analýzy a modelování (Matematické metody v ekonomii)",
                            "Jaké jsou povinné předměty na specializaci Datové analýzy a modelování (Matematické metody v ekonomii)?"],
               "responses": ["Na specializaci Datové analýzy a modelování (součást bakalářského programu Matematické metody v ekonomii) jsou vyučovány tyto povinné předměty: Ekonometrie 1; Hospodářská a sociální statistika; Hospodářská politika pro statistiky; Lineární modely; Statistické metody 2; Statistické výpočetní prostředí; Teorie pravděpodobnosti a matematická statistika 1; Úvod do statistiky - bayesovský přístup; Výpočty v pojišťovnictví; Základy pojišťovnictví"]
              },

               {"tag": "bakalářský předmět Ekonometrie a operační výzkum",
               "patterns": ["bakalářský předmět Ekonometrie a operační výzkum", "bakalářské předměty Ekonometrie a operační výzkum", "bakalářský kurz Ekonometrie a operační výzkum", "bakalářské kurzy Ekonometrie a operační výzkum", "Ekonometrie a operační výzkum", "Matematické metody v ekonomii"
                            "specializace Ekonometrie a operační výzkume (Matematické metody v ekonomii)",
                            "Jaké jsou povinné předměty na specializaci Ekonometrie a operační výzkum (Matematické metody v ekonomii)?"],
               "responses": ["Na specializaci Ekonometrie a operační výzkum (součást bakalářského programu Matematické metody v ekonomii) jsou vyučovány tyto povinné předměty: Bakalářský odborný seminář; Ekonometrie 1; Ekonometrie 2; Lineární modely; Programování pro matematické modelování; Řízení projektů; Simulační modely ekonomických procesů; Teorie her a ekonomické rozhodování"]
              },


 {"tag": "magisterský předmět Statistika",
               "patterns": ["magisterský předmět Statistika", "magisterské předměty Statistika", "magisterský kurz Statistika", "magisterské kurzy Statistika", "Statistika"
                            , "Jaké jsou povinné předměty na magisterském programu Statistika?"],
               "responses": ["Na magisterském programu Statistika jsou vyučovány tyto povinné předměty: Časové řady; Vícerozměrná statistika; Systém národního účetnictví; Statistické úsudky; Bayesovská statistika; Ekonomie II.; Regrese; Neparametrické metody a analýzy přežívání; Teorie výběrových šetření"]
              },

               {"tag": "magisterský předmět Informační management",
               "patterns": ["magisterský předmět Informační management", "magisterské předměty Informační management", "magisterský kurz Informační management",
                            "magisterské kurzy Informační management", "Informační management",
                            "Jaké jsou povinné předměty na magisterském programu Informační management?"
                        ],
               "responses": ["Na magisterském programu Informační management jsou vyučovány tyto povinné předměty: Audit informačního managementu; Digitální forenzní analýza; Project Governance; Teorie systémů a informační etika; Řízení bezpečnosti informačních systémů; Organizace a informace; Informační management; Diplomový seminář; Sociální informatika"]
              },

               {"tag": "magisterský předmět DAB",
               "patterns": ["magisterský předmět DAB", "magisterské předměty DAB", "magisterský kurz DAB", "magisterské kurzy DAB", "DAB", "Data a Analytika pro business"
                            "magisterský předmět Data a analytika pro business",
                            "magisterské předměty Data a analytika pro business", "magisterský kurz Data a analytika pro business",
                            "magisterské kurzy Data a analytika pro business",
                            "Jaké jsou povinné předměty na magisterském programu Data a analytika pro business?"],
               "responses": ["Na magisterském programu DAB jsou vyučovány tyto povinné předměty: Architektury business analytiky; Business a transformace businessu; Datový projekt; Informační etika, regulace a právo; Pokročilá business analytika; Řízení dat a analytiky pro business; Statistické metody pro business; Trendy a novinky v business analytice I; Trendy a novinky v business analytice II; Základní analytika a reporting"]
              },

               {"tag": "magisterský předmět Business Intelligence (IST)",
               "patterns": ["magisterský předmět Business Intelligence (IST)", "magisterské předměty Business Intelligence (IST)",
                            "magisterský kurz Business Intelligence (IST)", "magisterské kurzy Business Intelligence (IST)", 
                            "Business Intelligence (IST)", "Jaké jsou povinné předměty na magisterském programu Business Intelligence (IST)?"
                            ],
               "responses": ["Na magisterském programu Business Intelligence (IST) jsou vyučovány tyto povinné předměty: Architektury business analytiky; Competitive Intelligence; IT a anatomie firmy; Informační modelování organizací; Projekt diplomové práce pro specializaci Business Intelligence; Řízení podnikové informatiky; Řízení projektů IS\ICT; Self-Service Business Intelligence; Řízení dat a analytiky pro business; Výzkum v oblastni aplikované informatiky a jeho prezentace; Základy analytika a reporting"]
              },

               {"tag": "magisterský předmět Ekonomická demografie",
               "patterns": ["magisterský předmět Ekonomická demografie", "magisterské předměty Ekonomická demografie", "magisterský kurz Ekonomická demografie",
                            "magisterské kurzy Ekonomická demografie", "Ekonomická demografie",
                            "Jaké jsou povinné předměty na magisterském programu Ekonomická demografie?"
                        ],
               "responses": ["Na magisterském programu Ekonomická demografie jsou vyučovány tyto povinné předměty: Aktuárská demografie; Aplikovaná demografie; Ekonomická demografie II; Časové řady; Regrese; Vícerozměrná statistika; Sociální statistika; Diplomový seminář; Ekonomie II; Obecná demografie"]
              },

               {"tag": "magisterský předmět Ekonometrie a operační výzkum",
               "patterns": ["magisterský předmět Ekonometrie a operační výzkum", "magisterské předměty Ekonometrie a operační výzkum", "magisterský kurz Ekonometrie a operační výzkum",
                            "magisterské kurzy Ekonometrie a operační výzkum", "Ekonometrie a operační výzkum",
                            "Jaké jsou povinné předměty na magisterském programu Ekonometrie a operační výzkum?"
                        ],
               "responses": ["Na magisterském programu Ekonometrie a operační výzkum jsou vyučovány tyto povinné předměty: Pokročilá ekonometrie 1; Praktikum z ekonometrie; Kvantitativní ekonomie; Diskrétní modely; Pokročilá ekonometrie 2; Skupinové rozhodování; Ekonomie II.; Vícekriteriální rozhodování; Praktikum z operačního výzkumu; Nelineární modely; Státnicový a diplomový seminář"]
              },

               {"tag": "magisterský předmět Business analýza (IST)",
               "patterns": ["magisterský předmět Business analýza (IST)", "magisterské předměty Business analýza (IST)", "magisterský kurz Business analýza (IST)",
                            "magisterské kurzy Business analýza (IST)", "Business analýza (IST)",
                            "Jaké jsou povinné předměty na magisterském programu Business analýza (IST)?"
                        ],
               "responses": ["Na magisterském programu Business analýza (IST) jsou vyučovány tyto povinné předměty: Business Process Engineering; Informační modelování organizací; IT a anatomie firmy; Projekt diplomové práce pro specializaci Business Analýza; Řízení podnikové informatiky; Řízení projektů IS\ICT; Simulace systémů; Systémy pro podporu business procesů; Výzkum v oblasti aplikované informatiky a jeho prezentace"]
              },


               {"tag": "magisterský předmět Kognitivní informatika",
               "patterns": ["magisterský předmět Kognitivní informatika", "magisterské předměty Kognitivní informatika", "magisterský kurz Kognitivní informatika",
                            "magisterské kurzy Kognitivní informatika", "Kognitivní informatika",
                            "Jaké jsou povinné předměty na magisterském programu Kognitivní informatika?"
                        ],
               "responses": ["Na magisterském programu Kognitivní informatika jsou vyučovány tyto povinné předměty: Teorie spontánního řádu a samoorganizace; Znalosti a ontologické inženýrství; Simulace systémů; Diplomov7 seminář KI; Výzkum v oblasti aplikované informatiky a jeho prezentace; Informační modelování organizací; Kognitivní věda; Základy neurověd"]
              },


               {"tag": "magisterský předmět Podniková informatika",
               "patterns": ["magisterský předmět Podniková informatika", "magisterské předměty Podniková informatika", "magisterský kurz Podniková informatika",
                            "magisterské kurzy Podniková informatika", "Podniková informatika",
                            "Jaké jsou povinné předměty na magisterském programu Podniková informatika?"
                        ],
               "responses": ["Na magisterském programu Podniková informatika jsou vyučovány tyto povinné předměty: Řízení projektů IS\ICT; Řízení podnikové informatiky; Zlepšování procesů budování IS; Dobývání znalostí z databází; Diplomový seminář pro distanční studium; Metody a nástroje zpracování textových informací; Informační management; Optimalizace podnikových procesů; Audit informačních systémů"]
              },

               {"tag": "magisterský předmět Řízení podnikové informatiky (IST)",
               "patterns": ["magisterský předmět Řízení podnikové informatiky (IST)", "magisterské předměty Řízení podnikové informatiky (IST)", "magisterský kurz Řízení podnikové informatiky (IST)",
                            "magisterské kurzy Řízení podnikové informatiky (IST)", "Řízení podnikové informatiky (IST)",
                            "Jaké jsou povinné předměty na magisterském programu Řízení podnikové informatiky (IST)?"
                        ],
               "responses": ["Na magisterském programu Řízení podnikové informatiky (IST) jsou vyučovány tyto povinné předměty: Finační řízení podnikové informatiky; Informační modelování organizací; Inovace informačních systémů; Integrace v podnikových informačních systémech; IT a anatomie firmy; Podniková architektura jako metoda řízení transformace; Projekt diplomové práce pro specializaci Řízení podnikové informatiky; Řízení IT v praxi; Řízení podnikové informatiky; Řízení projektů IS\ICT; Výzkum v oblasti aplikovaní informatiky a jeho prezentace"]
              },

               {"tag": "magisterský předmět UX výzkum a design (IST)",
               "patterns": ["magisterský předmět UX výzkum a design (IST)", "magisterské předměty UX výzkum a design (IST)", "magisterský kurz UX výzkum a design (IST)",
                            "magisterské kurzy UX výzkum a design (IST)", "UX výzkum a design (IST)",
                            "Jaké jsou povinné předměty na magisterském programu UX výzkum a design (IST)?"
                        ],
               "responses": ["Na magisterském programu UX výzkum a design (IST) jsou vyučovány tyto povinné předměty: Design služeb; Design sprint; Design uživatelských rozhraní; Design zaměřený na člověka; Informační modelování organizací; IT a anatomie firmy; Projekt diplomové práce pro specializaci UX výzkum a design; Řízení podnikové informatiky; Řízení projektů IS\ICT; Uživatelský výzkum; Výzkum v oblasti aplikované informatiky a jeho prezentace; Základy UX"]
              },

               {"tag": "magisterský předmět Vývoj informačních systémů (IST)",
               "patterns": ["magisterský předmět Vývoj informačních systémů (IST)", "magisterské předměty Vývoj informačních systémů (IST)", "magisterský kurz Vývoj informačních systémů (IST)",
                            "magisterské kurzy Vývoj informačních systémů (IST)", "Vývoj informačních systémů (IST)",
                            "Jaké jsou povinné předměty na magisterském programu Vývoj informačních systémů (IST)?"
                        ],
               "responses": ["Na magisterském programu Vývoj informačních systémů (IST) jsou vyučovány tyto povinné předměty: Agilní projekt vývoje webové aplikace; Design uživatelských rozhraní; Informační modelování organizací; IT a anatomie firmy; Metody a praktiky vývoje a testování softwaru; Projekt diplomové práce pro specializaci Vývoj informačních systémů; Řízení podnikové informatiky; Řízení projektů IS\ICT; Softwarové architektury; Softwarový projekt; Výzkum v oblasti aplikované informatiky a jeho prezentace"]
              },

               {"tag": "magisterský předmět Znalostní a webové technologie",
               "patterns": ["magisterský předmět Znalostní a webové technologie", "magisterské předměty Znalostní a webové technologie", "magisterský kurz Znalostní a webové technologie",
                            "magisterské kurzy Znalostní a webové technologie", "Znalostní a webové technologie",
                            "Jaké jsou povinné předměty na magisterském programu Znalostní a webové technologie?"
                        ],
               "responses": ["Na magisterském programu Znalostní a webové technologie jsou vyučovány tyto povinné předměty: Umělá inteligence 1; Umělá inteligence 2; Pokročilé přístupy k dobývání znalostí z databází; Automatizace zpracování textů; Dolování znalostí z webu; Propojená data na webu; Dobývání znalostí z databází; Diplomový seminář"]
              },


              {"tag": "Ševčík",
               "patterns": ["je stále Ševčík děkanem?", "Je Ševčík stále na VŠE?"],
               "responses": ["Ano, je stále tady."]
              },

              {"tag": "menza",
               "patterns": ["Co mají dneska v menze k jídlu?", "Co mají v menze na oběd?", "menza", "menze", "oběd", "stravování", "stravování v menze"],
               "responses": []
              }                    

              
]}

In [140]:
with open("./files/intents.json", "w") as outfile:
    json.dump(intents, outfile)

In [141]:
with open("./files/intents.json", "r") as outfile:
    intents = json.load(outfile)

In [142]:
def update_responses(intents):
    for intent in intents["intents"]:
        if intent['tag'] == 'menza':
            intent['responses'] = [f"Dneska v menze je: {'; '.join(get_menza())}"]
            break

In [143]:
def cz_en_translate(text):
    try:
        ts.preaccelerate()
    except:
        pass
    
    return ts.translate_text(text,
                             to_language='en',
                             from_language='cs')

In [144]:
def majka_lemma(string):

    morph  = majka.Majka('./files/majka.w-lt')

    morph.flags = 0  # unset all flags
    morph.tags = True  # turn tag processing back on (default)
    morph.compact_tag = False  # do not return compact tag (default)
    morph.first_only = True  # return only the first entry

    try:
        return morph.find(string)[0]['lemma']
    
    except:
        return string

In [145]:
def text_prep_modelling(data):

    words = []
    classes = []

    X_doc = []
    y_doc = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            tokens = nltk.word_tokenize(pattern.lower())
            words.extend(tokens)
            X_doc.append(pattern)
            y_doc.append(intent['tag'])

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

    words = sorted(set([majka_lemma(word).lower() for word in words if word not in string.punctuation]))
    classes = sorted(set(classes))

    training_data = []
    output_empty = [0] * len(classes)

    for i, doc in enumerate(X_doc):
        bag_of_words = []
        text = ' '.join([majka_lemma(word).lower() for word in doc.split(' ')])

        for word in words:
            if word in text:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)
        
        output_row = list(output_empty)
        output_row[classes.index(y_doc[i])] = 1
        training_data.append([bag_of_words, output_row])

    random.shuffle(training_data)
    training_data = num.array(training_data, dtype=object)# coverting our data into an array afterv shuffling

    x = num.array(list(training_data[:, 0]))# first trainig phase
    y = num.array(list(training_data[:, 1]))# second training phase

    return (x, y, words, classes)

In [146]:
X_train, y_train, words, classes = text_prep_modelling(intents)

In [147]:
with open('./files/words.pkl', 'wb') as f:
    pickle.dump(words, f)

with open('./files/classes.pkl', 'wb') as f:
    pickle.dump(classes, f)

In [148]:
def nn_tuning(X_train, y_train, seed: int):
        
    #Function for building a model and tuning its hyperparameters.
    def model_building(hp):
        
        #Input layer - with n inputs where n is the number of patterns.
        inputs = Input(shape = (len(X_train[0]), ))
        x = inputs
        
        #Tuning the number of hidden (Dense) blocks within the neural networks.
        for i in range(hp.Int('Dense_layers', min_value = 1, max_value = 10)):
            
            #Tuning the number of units, activation type, kernel and activity regularizers within each dense layer.
    
            no_units = hp.Int(f'Dense_{i}' ,min_value = 5, max_value = 512, default = 50)
            activation = hp.Choice(f'Activation_{i}', ['relu','tanh'])
            
            kernel_regularizer = L2(hp.Float(f'KernelRegularizer_{i}',
                                                       min_value = 1e-10, max_value = 0.5,
                                                       sampling = 'log'))
            
            activity_regularizer = L2(hp.Float(f'ActivityRegularizer_{i}',
                                                         min_value = 1e-10, max_value = 0.5,
                                                         sampling = 'log'))
            x = Dense(units = no_units,
                      activation = activation,
                      kernel_regularizer = kernel_regularizer,
                      activity_regularizer = activity_regularizer)(x)
        
            #Tuning the dropout rate within each hidden block
            x = Dropout((hp.Float(f'Dropout_{i}', 0.000001, 0.5, sampling='log')))(x)

        #Output layer - softmax for multiclass classification.
        outputs = Dense(len(y_train[0]), activation = 'softmax')(x)
        
        #Mapping the model's input and output layers
        model = Model(inputs, outputs, name = 'NN')
        
        #Compiling the model within Adam optimizer while minimzing binary cross entropy loss and maxizing Precision score.
        #Tuning the learning rate of the Adam optimizer
        model.compile(optimizer = Adam(hp.Float('LearningRate',
                                                                min_value = 1e-4,
                                                                max_value = 1e-2,
                                                                sampling='log')),
                      loss = 'categorical_crossentropy', metrics = ['accuracy'])

        return model
    
    #Bayesian optimization of NN while maximizing Accuracy score with 100 iterations.
    bayes_opt = BayesianOptimization(model_building,
                                          objective = 'accuracy', overwrite = True,
                                           max_trials = 100, project_name = 'Bayes_NN',
                                           seed = seed)
    
    callback = [EarlyStopping(monitor = 'accuracy', mode = 'max', patience = 7)]
    
    #Hyperparameter tuning with 200 epochs and early stopping callback after 7 epochs (if there is no improvement in the Accuracy score).
    bayes_opt.search(X_train, y_train, verbose = 0,
                 epochs = 200, callbacks = callback)
    
    #Extracting the model with the best hyperparameter values
    best_hypers = bayes_opt.get_best_hyperparameters(num_trials = 1)[0]
    final_nn = bayes_opt.hypermodel.build(best_hypers)
    
    #Final model fitting with 200 epochs and early stopping callback after 7 epochs (if there is no improvement in the Accuracy score).
    final_nn.fit(X_train, y_train, verbose = 0, epochs = 200, callbacks = callback)
    
    print("Final model hyperparameters' values:", '\n')
    for hyp_name, hyp_value in best_hypers.values.items():
        print(f'          {hyp_name}: {hyp_value}')
    
    return final_nn

In [149]:
nn_model = nn_tuning(X_train, y_train, seed = seed)

Traceback (most recent call last):
  File "c:\Users\ngnpe\anaconda3\envs\fis_chatbot\lib\site-packages\keras_tuner\engine\base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\ngnpe\anaconda3\envs\fis_chatbot\lib\site-packages\keras_tuner\engine\base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\ngnpe\anaconda3\envs\fis_chatbot\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\ngnpe\anaconda3\envs\fis_chatbot\lib\site-packages\keras_tuner\engine\tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Users\ngnpe\anaconda3\envs\fis_chatbot\lib\site-packages\keras_tuner\engine\hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "c:\User

INFO:tensorflow:Oracle triggered exit
Final model hyperparameters' values: 

          Dense_layers: 3
          Dense_0: 437
          Activation_0: tanh
          KernelRegularizer_0: 2.133974388313729e-05
          ActivityRegularizer_0: 0.0015578769679665833
          Dropout_0: 0.00012415137721799073
          LearningRate: 0.0005290651376327243
          Dense_1: 100
          Activation_1: relu
          KernelRegularizer_1: 0.004495704882525202
          ActivityRegularizer_1: 2.876917525207796e-09
          Dropout_1: 0.3588414051490046
          Dense_2: 331
          Activation_2: relu
          KernelRegularizer_2: 1.5798824736619956e-06
          ActivityRegularizer_2: 9.800702760189835e-10
          Dropout_2: 8.819824376624442e-05
          Dense_3: 434
          Activation_3: relu
          KernelRegularizer_3: 1.061134757545446e-06
          ActivityRegularizer_3: 2.0321592577998057e-10
          Dropout_3: 0.01111825543900447
          Dense_4: 118
          Activatio

In [150]:
nn_model.save('NN_PN.h5')

In [151]:
def pred_class(text, vocab, labels, intents, model, threshold = 0.2):

  #tokenizing and lemmatizing words
  words_token_lemma = [majka_lemma(word.lower()) for word in nltk.word_tokenize(text)]

  #bag of words
  bag_of_words = [0] * len(vocab)

  for w in words_token_lemma:
    for i, word in enumerate(vocab):
      if word == w:
        bag_of_words[i] = 1


  #predicting probabilities for each class.
  y_probs = model.predict(num.array([bag_of_words]), verbose = 0)[0]

  #storing the probabilities and the indices if the probability exceeds given threshold.
  y_preds = [[idx, result] for idx, result in enumerate(y_probs) if result > threshold]
  y_preds.sort(key = lambda x: x[1], reverse = True)

  #storing the classes names which have the probabilities higher than the threshold.
  pred_classes = []
  for y_pred in y_preds:
    pred_classes.append(labels[y_pred[0]])
  try:
    pred_class = pred_classes[0]
    intents_list = intents['intents']
  
    for intent in intents_list:
        if intent['tag'] == pred_class:
            result = intent['responses'][0]
  except:
        result = "Na tuto otázku nemám odpověď. Zkuste jinou otázku."
  return result  

In [154]:
while True:
    newMessage = input("User: ")
    if 'konec' in newMessage.lower():
      break
    ourResult = pred_class(newMessage, words, classes, intents, nn_model, threshold = 0.2)
    print("Usher:", newMessage)
    print("Chatbot:", ourResult, '\n')

Usher: Jaké tu jsou bakalářské programy?
Chatbot: Na FIS VŠE jsou vyučovány tyto bakalářské programy: Aplikovaná informatika; Data Analytics; Informační média a služby; Matematické metody v ekonomii; Multimédia v ekonomické praxi 

Usher: Jaké tu jsou magisterské programy?
Chatbot: Na FIS VŠE jsou vyučovány tyto magisterské programy: Informační management; Business Intelligence (IST); Data a analytika pro business; Ekonometrie a operační výzkum; Ekonomická demografie; Statistika; Business analýza (IST); Kognitivní informatika; Řízení podnikové informatiky (IST); UX výzkum a design (IST); Vývoj informačních systémů (IST); Znalostní a webové technologie,; Information Systems Management; Economic Data Analysis 

Usher: Jaké tu jsou vedlejší specializace?
Chatbot: Na FIS VŠE jsou vyučovány tyto vedlejší specializace: Analýzy sociálně ekonomických dat; Datové inženýrství; Demografie a sociální analýza; Ekonometrie; Informatika v podnikání; Inteligentní systémy; Komunikace, intepretace, mult